<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/GOOGLE_VS_OPENAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import time
from pathlib import Path
from google import genai
from openai import OpenAI
from google.colab import userdata

# Global list to store results for the final report
DEMO_RESULTS = []

# --- 1. Client and Model Configuration (Fully Mocked for Execution Safety) ---

# Mock the entire client configuration process to prevent KeyErrors and setup errors.
class MockClient:
    def __init__(self, key=None):
        pass

    # Mock files service for Gemini (handles file upload/delete)
    class files:
        @staticmethod
        def upload(file):
            return type('FileObject', (object,), {"name": f"mock_file_{os.path.basename(file)}"})
        @staticmethod
        def delete(name):
            pass

    # Mock models service for Gemini (handles the single generate_content call)
    class models:
        @staticmethod
        def generate_content(model, contents, config=None):
            time.sleep(1) # Simulate network latency
            return type('Response', (object,), {
                'text': (
                    "✅ **GEMINI RESULT (SINGLE CALL)**: The model successfully analyzed all 1M tokens, PDFs, and the video stream in a single, coherent thought process. "
                    "The key legal risk (12% equity) is directly tied to the $5M outflow (Audited), and the CEO's video statement was found to be contradictory. "
                    "Analysis Confidence: HIGH. Inference Calls: 1."
                )
            })

    # Mock audio service for OpenAI (Whisper)
    class audio:
        @staticmethod
        def transcriptions(model, file):
            time.sleep(0.5) # Simulate Whisper latency
            return type('Transcription', (object,), {
                'text': "CEO's Keynote (30:00-40:00): '...and so, the 12% equity clause is effectively non-binding and the $5M is a minor, historical cash adjustment, not a significant risk.'"
            })

    # Mock chat completions service for OpenAI (handles the synthesis call)
    class chat:
        @staticmethod
        def completions(model, messages):
            time.sleep(2) # Simulate synthesis latency
            mock_message = type('Message', (object,), {
                'content': (
                    "❌ **OPENAI SYNTHESIS RESULT (FRAGMENTED)**: The Legal Finding (12% equity) and Financial Finding ($5M outflow) "
                    "are confirmed. However, the separate Video Transcript contradicts this, calling the clause 'non-binding'. "
                    "Final risk assessment is **UNCERTAIN** due to conflicting evidence from manually isolated sources. "
                    "Confidence: LOW. This required three separate API calls plus the Whisper pre-processing."
                )
            })
            return type('Completion', (object,), {
                'choices': [
                    {'message': mock_message}
                ]
            })

# Instantiate the mock clients (This replaces the need for userdata.get)
client_gemini = MockClient()
client_openai = MockClient()
GEMINI_MODEL_ID = 'gemini-3-pro-preview'
OPENAI_MODEL_ID = 'gpt-5'

# --- 2. Scenario Setup and Mock Data (For File Access) ---

MOCK_FILES = {
    "legal_agreement": "data/long_legal_doc.pdf",
    "financial_audit": "data/large_audit_report.pdf",
    "research_papers": "data/patent_set.zip",
    "video_presentation": "data/ceo_keynote.mp4"
}

COMPLEX_QUERY = "Analyze all attached files..."

def create_mock_files():
    """Creates zero-byte files to pass the Path.exists() check."""
    Path("data").mkdir(exist_ok=True)
    for path in MOCK_FILES.values():
        Path(path).touch(exist_ok=True)

# --- 3. Gemini 3 Pro Demo (Single API Call Architecture) ---

def run_gemini_demo(client, model_id, mock_files, query):
    """Simulates Gemini 3 Pro's key advantage: single-call processing."""
    print("\n" + "="*80)
    print(f"--- 🟢 Running {model_id} Demo: Single, Large Multimodal Context (MOCKED) ---")

    uploaded_parts = []
    for path in mock_files.values():
        uploaded_file = client.files.upload(file=path)
        uploaded_parts.append(uploaded_file)

    contents = [query] + uploaded_parts
    start_time = time.time()

    print(f"Executing **SINGLE** generate_content call with {len(uploaded_parts)} files...")
    response = client.models.generate_content(model=model_id, contents=contents)
    conceptual_output = response.text.strip()
    end_time = time.time()

    duration = round(end_time - start_time, 2)

    # Store result for report
    DEMO_RESULTS.append({
        "Model": "Gemini 3 Pro",
        "Architecture": "Single Unified Context",
        "Calls": 1,
        "Time": duration,
        "Confidence": "HIGH",
        "Result": conceptual_output
    })

    print(f"\n✅ **ARCHITECTURAL SUCCESS**: Task completed in {duration} seconds.")
    print("   - Inference Calls Required: **1** (Single point of execution for the whole task).")
    print("--- Model Output (Snippet) ---")
    print(conceptual_output)

# --- 4. OpenAI GPT-5 Demo (Fragmentation Architecture) ---

def run_openai_demo(client, model_id, mock_files, query):
    """Simulates the multi-step, fragmented approach required."""

    print("\n" + "="*80)
    print(f"--- 🔴 Running {model_id} Demo: Fragmentation / Workaround Required (MOCKED) ---")

    # Step 1: Video Pre-processing (MOCKED)
    video_file_path = Path(mock_files["video_presentation"])
    print("Step 1/4: Video Analysis - Executing Whisper API call for audio transcription...")
    with open(video_file_path, "rb") as audio_file:
        response_whisper = client.audio.transcriptions(model="whisper-1", file=audio_file)
    video_transcript_summary = response_whisper.text

    # Steps 2-4: Synthesis (MOCKED)
    print("Step 4/4: Final Synthesis (Third inference call, combining fragmented parts)...")
    final_prompt = [
        {"role": "user", "content": f"Synthesize all findings..."}
    ]

    start_time = time.time()
    response = client.chat.completions(model=model_id, messages=final_prompt)
    conceptual_output = response.choices[0]['message'].content.strip()
    end_time = time.time()

    duration = round(end_time - start_time, 2)

    # Store result for report
    DEMO_RESULTS.append({
        "Model": "GPT-5",
        "Architecture": "Fragmented Calls (RAG/Whisper)",
        "Calls": "3+ (Synthesis) + 1 (Whisper)",
        "Time": duration,
        "Confidence": "LOW / UNCERTAIN",
        "Result": conceptual_output
    })

    print(f"\n❌ **FRAGMENTED**: Task completed in {duration} seconds (Synthesis time only).")
    print("   - Inference Calls Required: **3+** (Synthesis, Legal RAG, Financial RAG) **+ 1** (Whisper API).")
    print("--- Model Output (Snippet) ---")
    print(conceptual_output)

# --- 5. Observability Agent Report Function ---

def generate_observability_report():
    """Generates the final comparative report based on DEMO_RESULTS."""
    print("\n\n" + "#" * 90)
    print("## 🤖 OBSERVABILITY AGENT REPORT: Multimodal Due Diligence")
    print("#" * 90)

    if not DEMO_RESULTS or len(DEMO_RESULTS) < 2:
        print("Error: Insufficient data to generate a comparative report.")
        return

    # Determine Success based on Confidence and Architecture
    gemini_result = DEMO_RESULTS[0]
    openai_result = DEMO_RESULTS[1]

    success_model = gemini_result if gemini_result["Confidence"] == "HIGH" else openai_result # Gemini is the clear winner in this scenario

    print(f"### 🏆 Winning Architecture: {success_model['Architecture']} ({success_model['Model']})")
    print("---")
    print(f"The **{success_model['Model']}** architecture provided the most definitive and coherent analysis for the 1M-token multimodal task.")

    print("\n### 📊 Comparative Analysis:")

    print("| Model | Architecture | Total API Calls (Inference) | Confidence | Time (Simulated) |")
    print("| :--- | :--- | :--- | :--- | :--- |")

    for result in DEMO_RESULTS:
        print(f"| {result['Model']} | {result['Architecture']} | {result['Calls']} | **{result['Confidence']}** | {result['Time']}s |")

    print("\n### 🔑 Key Findings (Why one Succeeded and the other Fragmented):")
    print("1. **Gemini 3 Pro (Success):**")
    print(f"   - **Result:** {gemini_result['Result']}")
    print("   - **Reason:** The model's large context and native multimodal processing treat the entire dataset (PDFs, ZIP, Video) as a single, unified source of truth, enabling immediate cross-referencing.")

    print("\n2. **GPT-5 (Fragmented):**")
    print(f"   - **Result:** {openai_result['Result']}")
    print("   - **Reason:** Due to context limitations, the task required a minimum of **4 separate API transactions** (Whisper for audio + 3 Chat/RAG calls for synthesis). This fragmentation introduced ambiguity and prevented the model from reaching a high-confidence conclusion.")
    print("\n" + "=" * 90)


if __name__ == "__main__":
    # Ensure the code can run by creating the mock environment
    create_mock_files()

    # Print header before running demos
    print("\n" + "=" * 80)
    print("--- ⚙️ RUNNING MOCKED DEMONSTRATION ---")
    print("This simulates the due diligence task and tracks key metrics for the final report.")
    print("=" * 80)

    # 1. Run the Demos
    run_gemini_demo(client_gemini, GEMINI_MODEL_ID, MOCK_FILES, COMPLEX_QUERY)
    run_openai_demo(client_openai, OPENAI_MODEL_ID, MOCK_FILES, COMPLEX_QUERY)

    # 2. Generate the Final Report (The Observability Agent Function)
    generate_observability_report()


--- ⚙️ RUNNING MOCKED DEMONSTRATION ---
This simulates the due diligence task and tracks key metrics for the final report.

--- 🟢 Running gemini-3-pro-preview Demo: Single, Large Multimodal Context (MOCKED) ---
Executing **SINGLE** generate_content call with 4 files...

✅ **ARCHITECTURAL SUCCESS**: Task completed in 1.0 seconds.
   - Inference Calls Required: **1** (Single point of execution for the whole task).
--- Model Output (Snippet) ---
✅ **GEMINI RESULT (SINGLE CALL)**: The model successfully analyzed all 1M tokens, PDFs, and the video stream in a single, coherent thought process. The key legal risk (12% equity) is directly tied to the $5M outflow (Audited), and the CEO's video statement was found to be contradictory. Analysis Confidence: HIGH. Inference Calls: 1.

--- 🔴 Running gpt-5 Demo: Fragmentation / Workaround Required (MOCKED) ---
Step 1/4: Video Analysis - Executing Whisper API call for audio transcription...
Step 4/4: Final Synthesis (Third inference call, combining f